In [121]:
import math
import pickle
import gzip
import numpy as np
import pandas as pd
from sklearn.utils import resample

import pandas as pd
import numpy as np
from keras.layers import Dense, Input, concatenate, Dropout, BatchNormalization, LSTM
from keras.models import Model, Sequential
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder,StandardScaler, Normalizer
import tensorflow as tf
import collections 

In [122]:
dfbands = pd.read_pickle('X_filtered.pkl')
# bands = pd.read_pickle('df_transformed.pkl')
dfbands.head(5)

variable,"b2__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.2","b3__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.2","b2__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.0",b2__mean_abs_change,b2__absolute_sum_of_changes,"b3__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.2","b2__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.0",b3__mean_abs_change,"b3__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.0",b3__absolute_sum_of_changes,...,"b4__linear_trend__attr_""intercept""","b2__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_9__w_10",b4__number_peaks__n_1,b2__autocorrelation__lag_7,"b1__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_14__w_2","b6__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_12__w_20",b3__c3__lag_2,b5__symmetry_looking__r_0.25,b5__symmetry_looking__r_0.30000000000000004,b1__symmetry_looking__r_0.30000000000000004
id,,,,,,,,,,,,,,,,,,,,,
615.0,519.504357,423.716195,647.078397,647.078397,36236.390207,273618.311367,447.784514,503.257899,503.257899,28182.442351,...,-309.031313,-845.660896,16.0,-0.051752,-105.344718,-554.608005,1.525137e+07,1.0,1.0,1.0
713.0,2.138922,2.120542,2.241918,2.241918,123.305483,8.156167,2.024384,2.051365,2.051365,112.825074,...,8.056757,14.107800,17.0,0.677033,3.142531,11.621645,-8.940954e+01,1.0,1.0,1.0
730.0,1.517284,2.015711,1.856014,1.856014,92.800712,16.361706,1.412415,1.888433,1.888433,94.421627,...,-2.154664,-1.488664,17.0,-0.043021,0.221550,-3.167655,2.138440e+02,1.0,1.0,1.0
745.0,6.382901,12.654187,8.632928,8.632928,466.178130,1877.644380,1.630198,9.276039,9.276039,500.906092,...,17.777045,-22.191762,15.0,-0.012770,-1.816087,50.504436,6.531625e+03,1.0,1.0,1.0
1124.0,3.390240,6.607588,2.810284,2.810284,157.375895,388.366325,1.517726,4.686253,4.686253,262.430156,...,-5.657373,1.054442,18.0,0.255567,0.507083,-18.002683,1.037487e+04,1.0,1.0,1.0


In [123]:
bs = ['b1','b2','b3','b4','b5','b6']
remaining_features = ['approximate_entropy', 'autocorrelation', 'c3', 'change_quantiles', #'absolute_sum_of_changes'
                     'cid_ce', 'count_above_mean', 'count_below_mean', 'energy_ratio_by_chunks', 
                     'fft_aggregated', 'fft_coefficient', #'last_location_of_maximum',   #
                     'longest_strike_above_mean', 'mean_abs_change', 'range_count', 'skewness'] #'quantile' or relative change in quantile?
listf = [] 
for i in bands.columns:
    for f in remaining_features:
        if f in i:
            listf.append(i)
dfbands_reduced = dfbands[listf]

In [124]:
dfMetaData = pd.read_csv("training_set_metadata.csv")
dfMerged = dfbands_reduced.merge(dfMetaData,left_on = "id",right_on = "object_id")
dfMerged = dfMerged.drop(["distmod","object_id"],axis = 1)
dfMerged.columns[-1]

'target'

In [125]:
def balance(df):     
    return resample(df,
                    replace=True, 
                    n_samples=2313,   #max target 90
                    random_state=123)

def bootstrap(df,cntindex):
    idcounts = df.iloc[:, cntindex].value_counts().to_frame('counts')
#     print(idcounts)
    df90 = df.loc[df.iloc[:,cntindex]==90]
    minority_class = [42, 65, 16, 15, 62, 88, 92, 67, 52, 95, 6, 64, 53]
    upsampled_list = [df90]  

    for m in minority_class:
        t = df.loc[df.iloc[:,cntindex]==m]
        upsampled_list.append(balance(t))
    df_upsampled = pd.concat([i for i in upsampled_list])
    
    return df_upsampled
 
dfMerged = bootstrap(dfMerged, -1)     

In [126]:
dfMerged.shape

(32382, 855)

In [127]:
num_hidden_neurons = 128
dropout = 0.25
epochs = 200

#preprocessing 
dfTrain, dfTest = train_test_split(dfMerged)

y_train = dfTrain["target"]
X_train = dfTrain.drop(["target"],axis = 1)

y_test = dfTest["target"]
X_test = dfTest.drop(["target"],axis = 1)

normalizer = Normalizer()
X_train = normalizer.fit_transform(X_train)
X_test = normalizer.transform(X_test)

#convert y to categorical 
unique_y = np.unique(y_train)
class_map = dict()
for i,val in enumerate(unique_y):
    class_map[val] = i
        
y_map = np.zeros((y_train.shape[0],))
y_map = np.array([class_map[val] for val in y_train])
y_categorical = to_categorical(y_map)

unique_y_test = np.unique(y_test)
class_map_test = dict()
for i,val in enumerate(unique_y_test):
    class_map_test[val] = i
        
y_map_test = np.zeros((y_test.shape[0],))
y_map_test = np.array([class_map_test[val] for val in y_test])
y_categorical_test = to_categorical(y_map_test)

#loss function
y_count = collections.Counter(y_map)
wtable = np.zeros((len(unique_y),))

for i in range(len(unique_y)):
    wtable[i] = y_count[i]/y_map.shape[0]
    
def mywloss(y_true,y_pred):  
    yc=tf.clip_by_value(y_pred,1e-15,1-1e-15)
    loss=-(tf.reduce_mean(tf.reduce_mean(y_true*tf.log(yc),axis=0)/wtable))
    return loss

In [ ]:
model = Sequential()

model.add(Dense(num_hidden_neurons,input_dim = X_train[0].shape[0],activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout))

model.add(Dense(14,activation = "softmax"))

model.compile(loss = mywloss,optimizer = "adam",metrics = ['acc'])

model.fit(X_train, y_categorical, batch_size = 128,validation_data=(X_test,y_categorical_test), epochs=epochs, verbose=1)

Train on 24286 samples, validate on 8096 samples
Epoch 1/200
24286/24286 [==============================] - 2s 91us/step - loss: 1.8720 - acc: 0.3837 - val_loss: 1.5339 - val_acc: 0.4967
Epoch 2/200
24286/24286 [==============================] - 1s 58us/step - loss: 1.4932 - acc: 0.5016 - val_loss: 1.3512 - val_acc: 0.5564
Epoch 3/200
24286/24286 [==============================] - 1s 58us/step - loss: 1.3460 - acc: 0.5523 - val_loss: 1.2382 - val_acc: 0.6005
Epoch 4/200
24286/24286 [==============================] - 1s 59us/step - loss: 1.2412 - acc: 0.5908 - val_loss: 1.1454 - val_acc: 0.6248
Epoch 5/200
24286/24286 [==============================] - 1s 59us/step - loss: 1.1545 - acc: 0.6201 - val_loss: 1.0617 - val_acc: 0.6682
Epoch 6/200
24286/24286 [==============================] - 1s 61us/step - loss: 1.0941 - acc: 0.6381 - val_loss: 1.0009 - val_acc: 0.6785
Epoch 7/200
24286/24286 [==============================] - 1s 61us/step - loss: 1.0311 - acc: 0.6604 - val_loss: 0.9398 - v

24286/24286 [==============================] - 1s 58us/step - loss: 0.4369 - acc: 0.8547 - val_loss: 0.4354 - val_acc: 0.8693
Epoch 60/200
24286/24286 [==============================] - 2s 63us/step - loss: 0.4323 - acc: 0.8535 - val_loss: 0.4185 - val_acc: 0.8746
Epoch 61/200
24286/24286 [==============================] - 1s 59us/step - loss: 0.4284 - acc: 0.8552 - val_loss: 0.4316 - val_acc: 0.8702
Epoch 62/200
24286/24286 [==============================] - 2s 65us/step - loss: 0.4301 - acc: 0.8535 - val_loss: 0.4164 - val_acc: 0.8756
Epoch 63/200
24286/24286 [==============================] - 2s 65us/step - loss: 0.4248 - acc: 0.8556 - val_loss: 0.4199 - val_acc: 0.8773
Epoch 64/200
24286/24286 [==============================] - 1s 60us/step - loss: 0.4284 - acc: 0.8538 - val_loss: 0.4181 - val_acc: 0.8715
Epoch 65/200
24286/24286 [==============================] - 2s 68us/step - loss: 0.4175 - acc: 0.8578 - val_loss: 0.4104 - val_acc: 0.8807
Epoch 66/200
24286/24286 [==============

Epoch 118/200
24286/24286 [==============================] - 1s 60us/step - loss: 0.3322 - acc: 0.8851 - val_loss: 0.3831 - val_acc: 0.8972
Epoch 119/200
24286/24286 [==============================] - 1s 56us/step - loss: 0.3341 - acc: 0.8832 - val_loss: 0.3806 - val_acc: 0.8967
Epoch 120/200
24286/24286 [==============================] - 1s 57us/step - loss: 0.3302 - acc: 0.8889 - val_loss: 0.3854 - val_acc: 0.8962
Epoch 121/200
24286/24286 [==============================] - 1s 57us/step - loss: 0.3312 - acc: 0.8864 - val_loss: 0.3942 - val_acc: 0.8917
Epoch 122/200
24286/24286 [==============================] - 1s 57us/step - loss: 0.3304 - acc: 0.8871 - val_loss: 0.3687 - val_acc: 0.9019
Epoch 123/200
24286/24286 [==============================] - 1s 57us/step - loss: 0.3304 - acc: 0.8873 - val_loss: 0.3907 - val_acc: 0.8951
Epoch 124/200
24286/24286 [==============================] - 2s 68us/step - loss: 0.3265 - acc: 0.8913 - val_loss: 0.3803 - val_acc: 0.8990
Epoch 125/200
24286/